In [1]:
## 2/ spustit trénování pouze na prvních 10K vzorcích - jaká je zhruba úspěšnost? kolikrát je každá z tříd v prvních 10K vzorcích zastoupena?

In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

num_classes = 10
input_shape = (28, 28, 1)


(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

#vybrani 10K trenovacich dat
x_train = x_train[:10000]
y_train = y_train[:10000]

#pocitadlo vyskytu trid
occurrences = [0]*10
for y in y_train:
    occurrences[y]+=1

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (10000, 28, 28, 1)
10000 train samples
10000 test samples


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [4]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/15
71/71 [==============================] - 2s 29ms/step - loss: 1.0922 - accuracy: 0.6913 - val_loss: 0.3183 - val_accuracy: 0.9040
Epoch 2/15
71/71 [==============================] - 2s 27ms/step - loss: 0.3194 - accuracy: 0.9046 - val_loss: 0.1989 - val_accuracy: 0.9460
Epoch 3/15
71/71 [==============================] - 2s 27ms/step - loss: 0.2171 - accuracy: 0.9348 - val_loss: 0.1569 - val_accuracy: 0.9520
Epoch 4/15
71/71 [==============================] - 2s 27ms/step - loss: 0.1743 - accuracy: 0.9489 - val_loss: 0.1294 - val_accuracy: 0.9600
Epoch 5/15
71/71 [==============================] - 2s 27ms/step - loss: 0.1501 - accuracy: 0.9560 - val_loss: 0.1156 - val_accuracy: 0.9710
Epoch 6/15
71/71 [==============================] - 2s 27ms/step - loss: 0.1292 - accuracy: 0.9603 - val_loss: 0.0974 - val_accuracy: 0.9770
Epoch 7/15
71/71 [==============================] - 2s 26ms/step - loss: 0.1141 - accuracy: 0.9658 - val_loss: 0.0961 - val_accuracy: 0.9740
Epoch 8/15
71

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
#uspesnost
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.06029188260436058
Test accuracy: 0.9800000190734863


In [6]:
for i, occ in enumerate(occurrences):
    print('Cislo {} se ve vzorcich vyskytuje {}.'.format(str(i), occ))

Cislo 0 se ve vzorcich vyskytuje 1001.
Cislo 1 se ve vzorcich vyskytuje 1127.
Cislo 2 se ve vzorcich vyskytuje 991.
Cislo 3 se ve vzorcich vyskytuje 1032.
Cislo 4 se ve vzorcich vyskytuje 980.
Cislo 5 se ve vzorcich vyskytuje 863.
Cislo 6 se ve vzorcich vyskytuje 1014.
Cislo 7 se ve vzorcich vyskytuje 1070.
Cislo 8 se ve vzorcich vyskytuje 944.
Cislo 9 se ve vzorcich vyskytuje 978.


[kitt] Fajn, ověřili jsme tedy, že třídy jsou v trénovací množině neseřazené, ale (prakticky asi náhodně) namíchané, což nám přineslo (skoro stejné) zastoupení všech tříd v prvních 10K vzorcích a ne tak významný pokles výsledné úspěšnosti klasifikace.